In [50]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('C:\\Users\LIPSITA_SENAPATI\Desktop\TEACHERS.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

,Teacher,vote_average,vote_count,id,specialisation,sp-id
0,A,7.7,5415,862,c,1
1,B,6.9,2413,8844,python,2
2,C,6.5,92,15602,java,3


In [54]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
data = pd.read_csv(r'C:/Users/LIPSITA_SENAPATI/Documents/ratings.csv', low_memory=False)

# Print the first three rows
data.head(3)

,id,sp-id,vote_average
0,862,1,7.7
1,8844,2,6.9
2,15602,3,6.5


In [55]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)


In [56]:
# Create two user-item matrices for training and testing data
train_data_matrix = train_data.as_matrix(columns = ['id', 'sp-id','vote_average'])
test_data_matrix = test_data.as_matrix(columns = ['id','sp-id','vote_average'])

c:\users\lipsita_senapati\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\lipsita_senapati\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])


[[1.         0.9999998  0.99999977 0.99999881]
 [0.9999998  1.         1.         0.99999763]
 [0.99999977 1.         1.         0.99999753]
 [0.99999881 0.99999763 0.99999753 1.        ]]


In [58]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

[[ 1.         -0.13695465 -0.40174309]
 [-0.13695465  1.          0.01655397]
 [-0.40174309  0.01655397  1.        ]]


In [59]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [60]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [61]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))


User-based CF RMSE: 11464.995405300666
Item-based CF RMSE: 12981.316938298321


In [62]:
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

User-based CF RMSE: 3527.5670666363553
Item-based CF RMSE: 3040.9510575289396
